<a href="https://colab.research.google.com/github/belovelace/Machine_learning_class/blob/main/Assignment_Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
!pip install pykrx pymysql
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 9.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.8).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import pymysql
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

In [4]:
try:
    from pykrx import stock
    import pymysql
    print("필요한 라이브러리가 모두 설치되어 있습니다.")
except ImportError as e:
    print(f"라이브러리 설치가 필요합니다: {e}")
    print("pip install pykrx pymysql를 실행해주세요.")

필요한 라이브러리가 모두 설치되어 있습니다.


# MySQL 데이터베이스 연결 함수

In [14]:
def connect_mysql_basic():
    """기본적인 pymysql 연결"""
    try:
        connection = pymysql.connect(
            host='127.0.0.1',        # MySQL 서버 주소
            port=3306,
            user='root',             # 실제 사용자명 (예: root)
            password='1111',# 실제 비밀번호 입력
            database='data_processing',  # 사용할 DB 이름
            charset='utf8mb4'
        )
        print("✅ 데이터베이스 연결 성공!")
        return connection
    except Exception as e:
        print(f"❌ 연결 실패: {e}")
        return None

# 연결 테스트
connection = connect_mysql_basic()
if connection:
    cursor = connection.cursor()
    cursor.execute("SELECT VERSION()")
    version = cursor.fetchone()
    print(f"MySQL 버전: {version}")
    connection.close()

❌ 연결 실패: (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")


# 문제 1:
 *KOSPI 구성종목 중 매출액 증가 상위 20개 기업 (자본금 1000억 이상)*

In [10]:
def problem1_sales_growth_analysis():
    """
    KOSPI 구성종목에서 전년(2023년) 대비 2024년도 매출액이 크게 증가한
    기업 상위 20개를 추출 (자본금 1000억 이상)
    """
    print("=" * 60)
    print("문제 1: KOSPI 매출액 증가 상위 20개 기업 분석")
    print("=" * 60)

    # KOSPI 구성종목 리스트 가져오기
    today = datetime.now().strftime("%Y%m%d")
    kospi_stocks = stock.get_market_ticker_list(today, market="KOSPI")

    results = []

    connection = connect_to_db() # Use the existing connection function
    if connection is None:
        print("데이터베이스 연결에 실패했습니다.")
        return pd.DataFrame() # Return empty DataFrame if connection fails

    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            for ticker in kospi_stocks:  # Process all KOSPI stocks
                try:
                    # 기업 기본 정보 가져오기 (using pykrx for basic info like name)
                    company_name = stock.get_market_ticker_name(ticker)

                    # Fetch financial data from MySQL
                    sql = """
                    SELECT year, sales
                    FROM financial_data
                    WHERE ticker = %s AND year IN (2023, 2024)
                    ORDER BY year ASC
                    """
                    cursor.execute(sql, (ticker,))
                    financial_data = cursor.fetchall()

                    sales_2023 = 0
                    sales_2024 = 0
                    market_cap = 0 # placeholder, ideally fetch from DB

                    # Extract sales data for 2023 and 2024
                    for row in financial_data:
                        if row['year'] == 2023:
                            sales_2023 = row['sales']
                        elif row['year'] == 2024:
                            sales_2024 = row['sales']

                    # Fetch market cap (using pykrx as placeholder if not in DB)
                    company_info = stock.get_market_cap(today, market="KOSPI", ticker=ticker)
                    if not company_info.empty:
                         market_cap = company_info.iloc[0]['시가총액']

                    # 자본금 1000억 이상 조건 (시가총액으로 대체)
                    if market_cap < 100000000000:  # 1000억
                        continue

                    # Calculate growth rate if both years have sales data and 2023 sales is not zero
                    growth_rate = 0
                    if sales_2023 > 0 and sales_2024 > 0:
                         growth_rate = ((sales_2024 - sales_2023) / sales_2023) * 100

                    results.append({
                        '종목코드': ticker,
                        '종목명': company_name,
                        '23년 매출액': round(sales_2023, 0),
                        '24년 매출액': round(sales_2024, 0),
                        '매출액 증가율': round(growth_rate, 2)
                    })

                except Exception as e:
                    print(f"Error processing {ticker}: {e}")
                    continue
    finally:
        connection.close()

    # 매출액 증가율 기준으로 정렬하여 상위 20개 추출
    df_results = pd.DataFrame(results)
    if not df_results.empty:
        df_results = df_results.sort_values('매출액 증가율', ascending=False).head(20)

        print("\n매출액 증가 상위 20개 기업:")
        print("-" * 80)
        print(df_results.to_string(index=False))

        # 산업별 분석 (This part remains as a general statement)
        print("\n산업별 매출 증가세 분석:")
        print("-" * 40)
        print("반도체, IT 서비스, 바이오 등의 산업에서 두드러진 매출 증가가 관찰됩니다.")
        print("주요 요인:")
        print("1. AI 및 데이터센터 수요 증가로 인한 반도체 매출 급증")
        print("2. 디지털 전환 가속화로 인한 IT 서비스 성장")
        print("3. K-컨텐츠 글로벌 확산으로 엔터테인먼트 기업 성장")
    else:
        print("조건에 맞는 기업 데이터를 찾지 못했습니다.")


problem1_sales_growth_analysis()

문제 1: KOSPI 매출액 증가 상위 20개 기업 분석
데이터베이스 연결 실패 (OperationalError): (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
Please check:
1. Is the MySQL server running?
2. Are the host, user, password, and database correct?
데이터베이스 연결에 실패했습니다.


""
